In [1]:
import pandas as pd
import glob
import os
import pathlib
import re

In [2]:
subdirectory = '00-raw/Wingspan_Raw/wingspan_nbaorg'
target_directory = pathlib.Path.cwd() / subdirectory
file_list = list(target_directory.glob('*.csv'))

print(f'datasets merged: {len(file_list)}')

df = pd.DataFrame()

for file in file_list:
    add_df = pd.read_csv(file)
    df = pd.concat([df, add_df], ignore_index = True)

print(df.shape)

datasets merged: 20
(1313, 10)


In [3]:
#rename columns: need name, height, wingspan - and for later, way to 
df.columns = df.columns.str.strip()

names = {
    'Crom_text__NpR1_':'name',
    'tablescraper-selected-row 4':'height',
    'tablescraper-selected-row 8':'wingspan'
}

df = df.rename(columns=names)
df = df[['name','height','wingspan']]
df = df.dropna(axis='index', how='any')

df.head(1)

,name,height,wingspan
0,Kenny Adeleke,6' 6.5'',6' 10.25''


In [4]:
#parsing strings for height calculations
def height_parser(str_in):
    height = re.findall(r'(\d{1,3}(?:\.\d+)?)', str_in)
    h_num = [float(i) for i in height]
    conversion_unit = 12 #inches
    inches = (h_num[0]*conversion_unit) + h_num[1]
    return inches


In [5]:
df['height_inches'] = df['height'].apply(height_parser)
df['wingspan_inches'] = df['wingspan'].apply(height_parser)
df['relative'] = df['wingspan_inches'] - df['height_inches']

df=df[['name','height_inches','wingspan_inches','relative']]

nbaorg_combined = df
#df.to_csv('01-interim/wingspan_cleaned/nbaorg_combined_cleaned.csv')
wingspan_nbaorg.head(1)

NameError: name 'wingspan_nbaorg' is not defined

In [6]:
#========= CLEANING 2025 WINGSPAN =========

In [7]:
file_name = 'wingspan_crafted_2025'
df = pd.read_csv(f'00-raw/Wingspan_Raw/{file_name}.csv')

df['relative'] = df['wingspan_inches'] - df['height_inches']
df=df[['name','height_inches','wingspan_inches','relative']]

df = df.dropna(axis='index', how='any')



wingspan_crafted_2025 = df
wingspan_crafted_2025.head(1)

#df.to_csv(f'01-interim/wingspan_cleaned/{file_name}_cleaned.csv')

,name,height_inches,wingspan_inches,relative
0,AJ Johnson,76.25,80.5,4.25


In [8]:
#========= CLEANING 2026 WINGSPAN =========
file_name = 'wingspan_crafted_2026'
df = pd.read_csv(f'00-raw/Wingspan_Raw/{file_name}.csv')
names = {
    'the-name':'name',
    'tablescraper-selected-row':'height',
    'tablescraper-selected-row 2':'wingspan',
    'tablescraper-selected-row 3':'relative'
}

df = df.rename(columns=names)
df = df[['name','height','wingspan','relative']]
df = df.dropna(axis='index', how='any')

#display(raw_df)
#display(df)

In [9]:
df['height_inches'] = df['height'].apply(height_parser)
df['wingspan_inches'] = df['wingspan'].apply(height_parser)
df=df[['name','height_inches','wingspan_inches','relative']]

wingspan_crafted_2026 = df
wingspan_crafted_2026.head(1)


#df.to_csv(f'01-interim/wingspan_cleaned/{file_name}_cleaned.csv')

,name,height_inches,wingspan_inches,relative
0,Mo Bamba,83.25,94.0,10.75


In [10]:
#WINGSPAN_KAGGLE
file_name = 'wingspan_Kaggle'
df = pd.read_csv(f'00-raw/Wingspan_Raw/{file_name}.csv')
names = {
    'PLAYER':'name',
    'HGT':'height_inches',
    'WNGSPN':'wingspan_inches'
}

df = df.rename(columns=names)
df = df[['name','height_inches','wingspan_inches']]
df['relative'] = df['wingspan_inches'] - df['height_inches']
df = df.dropna(axis='index', how='any')

wingspan_Kaggle = df
wingspan_Kaggle.head(1)
#df.to_csv(f'01-interim/wingspan_cleaned/{file_name}_cleaned.csv')

,name,height_inches,wingspan_inches,relative
0,"Almansa, Izan",81.25,85.75,4.5


In [11]:
#=========== FINISHED INDIVIDUAL PROC, NOW MERGE ==============

In [12]:
list_source = [nbaorg_combined,
               wingspan_crafted_2025,
               wingspan_crafted_2026,
               wingspan_Kaggle]


df_merge = pd.DataFrame()

for f in list_source:
    df_merge = df_merge.combine_first(f)

df_merge=df_merge[['name','height_inches','wingspan_inches','relative']]
print(df_merge)

df_merge.to_csv(f'01-interim/wingspan_cleaned/combined_cleaned.csv')



'''print(f'datasets merged: {len(file_list)}')

df = pd.DataFrame()

for file in file_list:
    add_df = pd.read_csv(file)
    df = pd.concat([df, add_df], ignore_index = True)

print(df.shape)'''

                   name  height_inches  wingspan_inches  relative
0         Kenny Adeleke          78.50            82.25      3.75
1          Maurice Ager          75.25            79.75      4.50
2     LaMarcus Aldridge          82.00            88.75      6.75
3         Morris Almond          76.75            82.00      5.25
4          Lou Amundson          79.20            83.50      4.30
...                 ...            ...              ...       ...
1673        Smith, Mike          78.50            80.50      2.00
1674  Stephens, Jarrett          77.25            81.25      4.00
1675   Thornton, Bootsy          75.50            78.50      3.00
1676      Walls, Jaquay          73.25            74.50      1.25
1677    Watkins, Jameel          81.50            90.00      8.50

[1677 rows x 4 columns]


"print(f'datasets merged: {len(file_list)}')\n\ndf = pd.DataFrame()\n\nfor file in file_list:\n    add_df = pd.read_csv(file)\n    df = pd.concat([df, add_df], ignore_index = True)\n\nprint(df.shape)"